In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(r"Online Retail.xlsx",encoding='latin-1')
df.shape

(541909, 8)

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [6]:
df = df[['CustomerID','Quantity', 'InvoiceDate','UnitPrice']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   CustomerID   406829 non-null  float64       
 1   Quantity     541909 non-null  int64         
 2   InvoiceDate  541909 non-null  datetime64[ns]
 3   UnitPrice    541909 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 16.5 MB


In [7]:
df.isnull().sum()

CustomerID     135080
Quantity            0
InvoiceDate         0
UnitPrice           0
dtype: int64

In [8]:
# dropping the rows where customerid is missing
df.dropna(inplace=True)

In [9]:
df.isnull().sum()

CustomerID     0
Quantity       0
InvoiceDate    0
UnitPrice      0
dtype: int64

In [10]:
df.shape

(406829, 4)

In [11]:
df.head()

,CustomerID,Quantity,InvoiceDate,UnitPrice
0,17850.0,6,2010-12-01 08:26:00,2.55
1,17850.0,6,2010-12-01 08:26:00,3.39
2,17850.0,8,2010-12-01 08:26:00,2.75
3,17850.0,6,2010-12-01 08:26:00,3.39
4,17850.0,6,2010-12-01 08:26:00,3.39


In [12]:
# calculating Recency, frequency, monetry
import datetime
cdate = datetime.datetime.now()
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

def get_recency(x):
    #print(cdate-x.max())
    return (cdate-x.max()).days

df_rec = df.groupby(['CustomerID'])['InvoiceDate'].apply(get_recency)
df_rec.name="Recency"

In [13]:
# frequency
df_freq = df.groupby("CustomerID")['CustomerID'].apply(len)
df_freq.name = "Frequency"

In [14]:
# Monetary
df['Monetary'] = df.Quantity * df.UnitPrice
df_mon = df.groupby('CustomerID')['Monetary'].apply(sum)

In [15]:
df2 = pd.merge(left=df_rec.reset_index(),right = df_freq.reset_index(),on="CustomerID")
df2 = pd.merge(left = df2, right=df_mon.reset_index(),on="CustomerID")
df2.head(10)

,CustomerID,Recency,Frequency,Monetary
0,12346.0,3838,2,0.00
1,12347.0,3515,182,4310.00
2,12348.0,3588,31,1797.24
3,12349.0,3532,73,1757.55
4,12350.0,3823,17,334.40
5,12352.0,3549,95,1545.41
6,12353.0,3717,4,89.00
7,12354.0,3745,58,1079.40
8,12355.0,3727,13,459.40
9,12356.0,3536,59,2811.43


In [16]:
df2.shape

(4372, 4)

In [17]:
df2['R'] = pd.qcut(df2.Recency,[0,0.25,0.50,0.75,1.0],labels = [1,2,3,4])
df2['F'] = pd.qcut(df2.Frequency,[0,0.25,0.50,0.75,1.0],labels = [4,3,2,1])
df2['M'] = pd.qcut(df2.Monetary,[0,0.25,0.50,0.75,1.0],labels = [4,3,2,1])
df2

,CustomerID,Recency,Frequency,Monetary,R,F,M
0,12346.0,3838,2,0.00,4,4,4
1,12347.0,3515,182,4310.00,1,1,1
2,12348.0,3588,31,1797.24,3,3,1
3,12349.0,3532,73,1757.55,2,2,1
4,12350.0,3823,17,334.40,4,4,3
...,...,...,...,...,...,...,...
4367,18280.0,3791,10,180.60,4,4,4
4368,18281.0,3693,7,80.82,4,4,4
4369,18282.0,3520,13,176.60,1,4,4
4370,18283.0,3516,756,2094.88,1,1,1


In [18]:
df2.CustomerID[(df2.R ==1) & (df2.F==1) & (df2.M==1)].to_csv("best_customers.csv")
df2.CustomerID[(df2.R ==3) & (df2.F==1) & (df2.M==1)].to_csv("almost_lost_customers.csv")
df2.CustomerID[(df2.R ==4) & (df2.F==1) & (df2.M==1)].to_csv("lost_customers.csv")